In [1]:
# Importing all the necessary libraries


import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files

!conda install -c conda-forge geopy --yes
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe


# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

# for webscraping import Beautiful Soup 
from bs4 import BeautifulSoup

import xml

!conda install -c conda-forge folium=0.5.0 --yes
import folium # map rendering library

print('Libraries imported.')

Solving environment: done

# All requested packages already installed.

Solving environment: done

# All requested packages already installed.

Libraries imported.


## Exracting the content from wiki page that has 'List of US Cities by population' in to a text file

In [2]:
link = 'https://en.wikipedia.org/wiki/List_of_United_States_cities_by_population'
page = requests.get(link) 
soup = BeautifulSoup(page.text)

### Finding the table that has the data that we need i.e. list of all cities with their population, Square Area, Location (coordinates)
 

In [3]:
table = soup.find_all('table')[4]

### Extracting the table from the webpage into a data frame by specifying the column names

In [4]:
table_rows = table.find_all('tr')
res = []
for tr in table_rows:
    td = tr.find_all('td')
    row = [tr.text.strip() for tr in td if tr.text.strip()]
    if row:
        res.append(row)
df = pd.DataFrame(res, columns=["Rank", "City", "State", "del1", "del2", "del3", "Sq.Area", "del5", "population density in Sq Mi", "Population density in Km2", "Location"])
df.head(25)

,Rank,City,State,del1,del2,del3,Sq.Area,del5,population density in Sq Mi,Population density in Km2,Location
0,1,New York[d],New York,"8,398,748","8,175,133",+2.74%,301.5 sq mi,780.9 km2,"28,317/sq mi","10,933/km2",40°39′49″N 73°56′19″W﻿ / ﻿40.6635°N 73.9387°W﻿...
1,2,Los Angeles,California,"3,990,456","3,792,621",+5.22%,468.7 sq mi,"1,213.9 km2","8,484/sq mi","3,276/km2",34°01′10″N 118°24′39″W﻿ / ﻿34.0194°N 118.4108°...
2,3,Chicago,Illinois,"2,705,994","2,695,598",+0.39%,227.3 sq mi,588.7 km2,"11,900/sq mi","4,600/km2",41°50′15″N 87°40′54″W﻿ / ﻿41.8376°N 87.6818°W﻿...
3,4,Houston[3],Texas,"2,325,502","2,100,263",+10.72%,637.5 sq mi,"1,651.1 km2","3,613/sq mi","1,395/km2",29°47′12″N 95°23′27″W﻿ / ﻿29.7866°N 95.3909°W﻿...
4,5,Phoenix,Arizona,"1,660,272","1,445,632",+14.85%,517.6 sq mi,"1,340.6 km2","3,120/sq mi","1,200/km2",33°34′20″N 112°05′24″W﻿ / ﻿33.5722°N 112.0901°...
5,6,Philadelphia[e],Pennsylvania,"1,584,138","1,526,006",+3.81%,134.2 sq mi,347.6 km2,"11,683/sq mi","4,511/km2",40°00′34″N 75°08′00″W﻿ / ﻿40.0094°N 75.1333°W﻿...
6,7,San Antonio,Texas,"1,532,233","1,327,407",+15.43%,461.0 sq mi,"1,194.0 km2","3,238/sq mi","1,250/km2",29°28′21″N 98°31′30″W﻿ / ﻿29.4724°N 98.5251°W﻿...
7,8,San Diego,California,"1,425,976","1,307,402",+9.07%,325.2 sq mi,842.3 km2,"4,325/sq mi","1,670/km2",32°48′55″N 117°08′06″W﻿ / ﻿32.8153°N 117.1350°...
8,9,Dallas,Texas,"1,345,047","1,197,816",+12.29%,340.9 sq mi,882.9 km2,"3,866/sq mi","1,493/km2",32°47′36″N 96°45′59″W﻿ / ﻿32.7933°N 96.7665°W﻿...
9,10,San Jose,California,"1,030,119","945,942",+8.90%,177.5 sq mi,459.7 km2,"5,777/sq mi","2,231/km2",37°17′48″N 121°49′08″W﻿ / ﻿37.2967°N 121.8189°...


### Finding the radius of each city with the help of Sq.Area, this step involves in preprocessing of the the column Sq.Area (changing its data type to float) then finding its square root

In [5]:
new= df["Sq.Area"].str.split("s", n=1, expand = True)
new = new[0].str.replace(u'\xa0',u'')
df["Sq.Area"] = new.str.replace(',','')
df["Sq.Area"] = df["Sq.Area"].astype(float)
df["Radius"] = np.sqrt(df["Sq.Area"])

### Drop unnecessary columns like Rank, del1, del2 ... del5, population density in Sq Mi from the data frame we extracted from the webpage


In [6]:
df.drop(columns = ["Rank", "del1", "del2", "del3", "del5", "Sq.Area", "population density in Sq Mi"], inplace = True)
df

,City,State,Population density in Km2,Location,Radius
0,New York[d],New York,"10,933/km2",40°39′49″N 73°56′19″W﻿ / ﻿40.6635°N 73.9387°W﻿...,17.363755
1,Los Angeles,California,"3,276/km2",34°01′10″N 118°24′39″W﻿ / ﻿34.0194°N 118.4108°...,21.649480
2,Chicago,Illinois,"4,600/km2",41°50′15″N 87°40′54″W﻿ / ﻿41.8376°N 87.6818°W﻿...,15.076472
3,Houston[3],Texas,"1,395/km2",29°47′12″N 95°23′27″W﻿ / ﻿29.7866°N 95.3909°W﻿...,25.248762
4,Phoenix,Arizona,"1,200/km2",33°34′20″N 112°05′24″W﻿ / ﻿33.5722°N 112.0901°...,22.750824
5,Philadelphia[e],Pennsylvania,"4,511/km2",40°00′34″N 75°08′00″W﻿ / ﻿40.0094°N 75.1333°W﻿...,11.584472
6,San Antonio,Texas,"1,250/km2",29°28′21″N 98°31′30″W﻿ / ﻿29.4724°N 98.5251°W﻿...,21.470911
7,San Diego,California,"1,670/km2",32°48′55″N 117°08′06″W﻿ / ﻿32.8153°N 117.1350°...,18.033303
8,Dallas,Texas,"1,493/km2",32°47′36″N 96°45′59″W﻿ / ﻿32.7933°N 96.7665°W﻿...,18.463477
9,San Jose,California,"2,231/km2",37°17′48″N 121°49′08″W﻿ / ﻿37.2967°N 121.8189°...,13.322913


### Splitting the cooridnates to Latitudes and Longitudes for each city

In [7]:
df["Location"]= df["Location"].str.split("/", n = 2, expand = True)[1]
df.head(25)

,City,State,Population density in Km2,Location,Radius
0,New York[d],New York,"10,933/km2",﻿40.6635°N 73.9387°W﻿,17.363755
1,Los Angeles,California,"3,276/km2",﻿34.0194°N 118.4108°W﻿,21.649480
2,Chicago,Illinois,"4,600/km2",﻿41.8376°N 87.6818°W﻿,15.076472
3,Houston[3],Texas,"1,395/km2",﻿29.7866°N 95.3909°W﻿,25.248762
4,Phoenix,Arizona,"1,200/km2",﻿33.5722°N 112.0901°W﻿,22.750824
5,Philadelphia[e],Pennsylvania,"4,511/km2",﻿40.0094°N 75.1333°W﻿,11.584472
6,San Antonio,Texas,"1,250/km2",﻿29.4724°N 98.5251°W﻿,21.470911
7,San Diego,California,"1,670/km2",﻿32.8153°N 117.1350°W﻿,18.033303
8,Dallas,Texas,"1,493/km2",﻿32.7933°N 96.7665°W﻿,18.463477
9,San Jose,California,"2,231/km2",﻿37.2967°N 121.8189°W﻿,13.322913


In [8]:
new = df["Location"].str.split(" ", n = 0, expand = False)
k = df.copy(deep = True)

In [9]:
Latitude = []
Longitude = []
for i in range(len(new)):
    Latitude.append(new[i][1][:-2])
    Longitude.append(new[i][2][:-3]) 

k["Latitude"] = Latitude
k["Longitude"] = Longitude
k["Latitude"] = k["Latitude"].str.replace(u'\ufeff',u'')
k.drop(columns = ["Location"], inplace = True)
k.head()
df = k.copy(deep = True)

In [10]:
df['Longitude'] = -df['Longitude'].astype(float)
df['Latitude'] = df['Latitude'].astype(float)
df['Radius'] = df['Radius']* 1000
df.head(25)

,City,State,Population density in Km2,Radius,Latitude,Longitude
0,New York[d],New York,"10,933/km2",17363.755354,40.6635,-73.9387
1,Los Angeles,California,"3,276/km2",21649.480363,34.0194,-118.4108
2,Chicago,Illinois,"4,600/km2",15076.471736,41.8376,-87.6818
3,Houston[3],Texas,"1,395/km2",25248.762346,29.7866,-95.3909
4,Phoenix,Arizona,"1,200/km2",22750.824161,33.5722,-112.0901
5,Philadelphia[e],Pennsylvania,"4,511/km2",11584.472366,40.0094,-75.1333
6,San Antonio,Texas,"1,250/km2",21470.910554,29.4724,-98.5251
7,San Diego,California,"1,670/km2",18033.302526,32.8153,-117.1350
8,Dallas,Texas,"1,493/km2",18463.477462,32.7933,-96.7665
9,San Jose,California,"2,231/km2",13322.912594,37.2967,-121.8189


## Getting the per capita income state wise for USA

In [11]:
link1 = 'https://en.wikipedia.org/wiki/List_of_United_States_counties_by_per_capita_income'
page1 = requests.get(link1) 
soup1 = BeautifulSoup(page1.text)
table = soup1.find_all('table')[1]
table_rows = table.find_all('tr')
res = []
for tr in table_rows:
    td = tr.find_all('td')
    row = [tr.text.strip() for tr in td if tr.text.strip()]
    if row:
        res.append(row)
df_state = pd.DataFrame(res, columns=["Rank", "Country-equivalent", "State", "Per capita income", "del2", "del3", "Population", "del5"])
df_state.head(25)

,Rank,Country-equivalent,State,Per capita income,del2,del3,Population,del5
0,1,New York County,New York,"$62,498","$69,659","$84,627","1,605,272","736,192"
1,2,Arlington,Virginia,"$62,018","$103,208","$139,244","214,861","94,454"
2,3,Falls Church City,Virginia,"$59,088","$120,000","$152,857","12,731","5,020"
3,4,Marin,California,"$56,791","$90,839","$117,357","254,643","102,912"
4,5,Alexandria City,Virginia,"$54,608","$85,706","$107,511","143,684","65,369"
5,6,Pitkin,Colorado,"$51,814","$72,745","$93,981","17,173","7,507"
6,7,Los Alamos,New Mexico,"$51,044","$106,686","$124,979","17,979","7,590"
7,8,Fairfax County,Virginia,"$50,532","$110,292","$128,596","1,101,071","389,908"
8,9,Hunterdon,New Jersey,"$50,349","$106,143","$125,828","127,047","46,816"
9,10,Borden,Texas,"$50,042","$71,607","$76,250",649,225


### Dropping unnecessary columns like Rank, del2, del3, del5 from the Table we extracted from the webpage that has population and percapita income

In [12]:
df_state.drop(columns = ['Rank','del2', 'del3', 'del5'], axis = 1, inplace = True)

In [13]:
df_state.head(25)

,Country-equivalent,State,Per capita income,Population
0,New York County,New York,"$62,498","1,605,272"
1,Arlington,Virginia,"$62,018","214,861"
2,Falls Church City,Virginia,"$59,088","12,731"
3,Marin,California,"$56,791","254,643"
4,Alexandria City,Virginia,"$54,608","143,684"
5,Pitkin,Colorado,"$51,814","17,173"
6,Los Alamos,New Mexico,"$51,044","17,979"
7,Fairfax County,Virginia,"$50,532","1,101,071"
8,Hunterdon,New Jersey,"$50,349","127,047"
9,Borden,Texas,"$50,042",649


### Plotting all the cities of USA that we have extracted from Wiki page, using their coordinates


In [14]:
# create map of USA cities that we have using latitude and longitude values
map_tohood = folium.Map(location=[37.0902,-95.7129], zoom_start=5)

# add markers to map
for lat, lng, state, city in zip(df['Latitude'], df['Longitude'], df['State'], df['City']):
    label = '{}, {}'.format(city, state)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='red',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.5,
        parse_html=False).add_to(map_tohood)  
    
map_tohood

## Define Four Square Credentials and version

In [39]:

CLIENT_ID = 'AN3SMQTKLWGT3NKKC3YDJI11QUXFOGDCP4LXFGPPZVVGGN21' #  Foursquare ID
CLIENT_SECRET = 'A3OP23GQ5C1GQXBLD3QE4TLONNW0XXHWS23BWEI4V5UNMNGL' #  Foursquare Secret
VERSION = '20190701'
LIMIT = 20
print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: AN3SMQTKLWGT3NKKC3YDJI11QUXFOGDCP4LXFGPPZVVGGN21
CLIENT_SECRET:A3OP23GQ5C1GQXBLD3QE4TLONNW0XXHWS23BWEI4V5UNMNGL


### Function that extracts necessary columns into a data frame from the json files that we get when we search using four square API out of the values that we have in the data frame that we got from the web page


In [44]:
def getNearbyVenues(names, latitudes, longitudes, radius):
    
    venues_list=[]
    for name, lat, lng,radius in zip(names, latitudes, longitudes,radius):
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
       # print(results)
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['City', 
                  'Latitude', 
                  'Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [46]:
df_venues = getNearbyVenues(names = df['City'], latitudes = df['Latitude'],longitudes = df['Longitude'], radius = df['Radius'])
df_venues.head()

,City,Latitude,Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,New York[d],40.6635,-73.9387,Brooklyn Botanic Garden,40.667622,-73.963191,Botanical Garden
1,New York[d],40.6635,-73.9387,Covenhoven,40.675143,-73.960203,Beer Bar
2,New York[d],40.6635,-73.9387,Brooklyn Museum,40.671521,-73.963677,Art Museum
3,New York[d],40.6635,-73.9387,Prospect Park Boathouse & Audubon Center,40.660884,-73.964949,Building
4,New York[d],40.6635,-73.9387,Kings Theatre,40.646110,-73.957175,Theater


In [47]:
k = df_venues.copy(deep = True)
weights_dict={'Movie Theater':4,'Beach':4,'Concert Hall':4,'Playground':4,'Coffee Shop':4,'Food Court':4,'Nightclub':4,'Toy / Game Store':4,'Theme Park Ride / Attraction':4,'Pub':4}
data = df_venues['Venue Category']
allVenues = data.astype("object", copy=True)

In [48]:
weights = []
for i in allVenues:
    if i in weights_dict.keys():
        weights.append(weights_dict[i])
    else :
        weights.append(0)
df_venues['weights'] = weights;
df_venues.head(25)

,City,Latitude,Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category,weights
0,New York[d],40.6635,-73.9387,Brooklyn Botanic Garden,40.667622,-73.963191,Botanical Garden,0
1,New York[d],40.6635,-73.9387,Covenhoven,40.675143,-73.960203,Beer Bar,0
2,New York[d],40.6635,-73.9387,Brooklyn Museum,40.671521,-73.963677,Art Museum,0
3,New York[d],40.6635,-73.9387,Prospect Park Boathouse & Audubon Center,40.660884,-73.964949,Building,0
4,New York[d],40.6635,-73.9387,Kings Theatre,40.646110,-73.957175,Theater,0
5,New York[d],40.6635,-73.9387,Prospect Park (Nethermead),40.660717,-73.968587,Field,0
6,New York[d],40.6635,-73.9387,Long Meadow,40.668758,-73.970304,Field,0
7,New York[d],40.6635,-73.9387,Prospect Park,40.661971,-73.971226,Park,0
8,New York[d],40.6635,-73.9387,Gold Star Beer Counter,40.675231,-73.966759,Bar,0
9,New York[d],40.6635,-73.9387,Grand Army Plaza Greenmarket,40.672526,-73.969673,Farmers Market,0


In [49]:
# Dropping the rows that we are not giving any weight
df_venues.drop(df_venues[df_venues.weights < 1.0].index, inplace=True)
df_venues.head(25)

,City,Latitude,Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category,weights
22,Los Angeles,34.0194,-118.4108,Blue Bottle Coffee,34.027115,-118.387637,Coffee Shop,4
30,Los Angeles,34.0194,-118.4108,Blue Bottle Coffee,33.980027,-118.408020,Coffee Shop,4
59,Chicago,41.8376,-87.6818,Sawada Coffee,41.883730,-87.648726,Coffee Shop,4
77,Houston[3],29.7866,-95.3909,White Oak Music Hall,29.785994,-95.367173,Concert Hall,4
99,Phoenix,33.5722,-112.0901,Dutch Bros. Coffee,33.509570,-112.074119,Coffee Shop,4
100,Philadelphia[e],40.0094,-75.1333,Amalgam Comics & Coffeehouse,39.985120,-75.124364,Coffee Shop,4
139,San Antonio,29.4724,-98.5251,Local Coffee,29.442251,-98.479625,Coffee Shop,4
157,San Diego,32.8153,-117.1350,Hawthorn Coffee,32.762778,-117.129704,Coffee Shop,4
221,Jacksonville[f],30.3369,-81.6616,Chamblin's Uptown,30.328552,-81.658890,Coffee Shop,4
224,Jacksonville[f],30.3369,-81.6616,Times-Union Center for the Performing Arts,30.325514,-81.662389,Concert Hall,4


### Copying only the relevants columns like city and weights to group all the venues by city and calculating the means for each city

In [50]:
citywise_venues_weights = df_venues[['City','weights']].copy()
citywise_venues_weights_means = citywise_venues_weights.groupby(['City']).mean()
citywise_venues_weights_means = citywise_venues_weights_means.reset_index(drop=False)
citywise_venues_weights_means.head()

,City,weights
0,Abilene,4
1,Alexandria[m],4
2,Allen,4
3,Amarillo,4
4,Anaheim,4


### Merging the table for which we calculated the means of weights city wise to the actual table that we got from the wiki page

In [51]:
city_selection = pd.merge(df, citywise_venues_weights_means, on='City')
city_selection = city_selection[['City','Population density in Km2','weights']].copy()
city_selection.head(25)

,City,Population density in Km2,weights
0,Los Angeles,"3,276/km2",4
1,Chicago,"4,600/km2",4
2,Houston[3],"1,395/km2",4
3,Phoenix,"1,200/km2",4
4,Philadelphia[e],"4,511/km2",4
5,San Antonio,"1,250/km2",4
6,San Diego,"1,670/km2",4
7,Jacksonville[f],455/km2,4
8,Fort Worth,962/km2,4
9,Columbus,"1,520/km2",4


In [52]:
# Preprocessing the population density in Km2 column as we have to normalize these values
k = city_selection.copy(deep = True)
k['Population density in Km2'] = k['Population density in Km2'].str.split("/", n = 0, expand = True)
k['Population density in Km2'] = k['Population density in Km2'].str.replace(',','')
k['Population density in Km2'] = k['Population density in Km2'].astype(float)
city_selection = k.copy(deep = True)
city_selection.head(25)

,City,Population density in Km2,weights
0,Los Angeles,3276.0,4
1,Chicago,4600.0,4
2,Houston[3],1395.0,4
3,Phoenix,1200.0,4
4,Philadelphia[e],4511.0,4
5,San Antonio,1250.0,4
6,San Diego,1670.0,4
7,Jacksonville[f],455.0,4
8,Fort Worth,962.0,4
9,Columbus,1520.0,4


#### Normalizing our data frame

In [53]:
# Normalizing the data frame
from sklearn import preprocessing
column_names_to_normalize = ['Population density in Km2', 'weights']
x = city_selection[column_names_to_normalize].values #returns a numpy array
min_max_scaler = preprocessing.MinMaxScaler()
x_scaled = min_max_scaler.fit_transform(x)
city_selection[column_names_to_normalize] = pd.DataFrame(x_scaled)
city_selection.head(25)

,City,Population density in Km2,weights
0,Los Angeles,0.470174,0.0
1,Chicago,0.664224,0.0
2,Houston[3],0.194489,0.0
3,Phoenix,0.165909,0.0
4,Philadelphia[e],0.651180,0.0
5,San Antonio,0.173238,0.0
6,San Diego,0.234794,0.0
7,Jacksonville[f],0.056720,0.0
8,Fort Worth,0.131027,0.0
9,Columbus,0.212810,0.0


#### Calculating the sum of normalized columns to determine the city that has maximum sum and conclude that one locality in that city would be the best fit

In [58]:
#calculating the sum of normalized columns to determine the city that has maximum sum and conclude that one locality in that city would be the best fit 
city_selection['sum'] = city_selection['Population density in Km2'] + city_selection['weights']
row_num = city_selection['sum'].argmax()
city_name = city_selection['City'].iloc[row_num]
city_name

/opt/conda/envs/Python36/lib/python3.6/site-packages/ipykernel/__main__.py:3: FutureWarning: 
The current behaviour of 'Series.argmax' is deprecated, use 'idxmax'
instead.
The behavior of 'argmax' will be corrected to return the positional
maximum in the future. For now, use 'series.values.argmax' or
'np.argmax(np.array(values))' to get the position of the maximum
row.
  app.launch_new_instance()


'Jersey City'

In [59]:
# Finding the state in which that city belongs
row = df.loc[df['City']== city_name].index[0]
state_name = df['State'].iloc[row]
state_name

'New Jersey'

### Jone Doe wondering what there income per capita (do they have the opportunity to go to his Best Bar)

In [60]:
# checking the percapita income of New Jercy 
p_row = df_state.loc[df_state['State'] == state_name].index[0]
per_capital_income = df_state['Per capita income'].iloc[p_row]
print("Per capita income of New Jercy is :", per_capital_income)

Per capita income of New Jercy is : $50,349


# Let`s find the location in Jersey City would be the best to open a Best Bar

In [61]:
# Getting coordinates of New Jersey
lat_newJercy = df['Latitude'].iloc[row]
long_newJercy = df['Longitude'].iloc[row]
print(lat_newJercy, long_newJercy)

40.7114 -74.0648


### Getting all the venues of New Jersey using four square API within our considered radius and Limits

In [63]:
# Getting the venues of New Jersey using four square API 

def getNearbyVenues1(name, latitudes, longitudes, radius):
    
    LIMIT = 150       
        # create the API request URL
    url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            latitudes, 
            longitudes, 
            radius, 
            LIMIT)
            
        # make the GET request
    results = requests.get(url).json()["response"]['groups'][0]['items']
   # print(results)
    venues_list=[]
    venues_list.append([(name,lat,lng,v['venue']['name'],v['venue']['location']['lat'],v['venue']['location']['lng'],v['venue']['categories'][0]['name'])for v in results])
    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['City', 'Latitude', 'Longitude', 'Venue', 'Venue Latitude', 'Venue Longitude','Venue Category']
    return(nearby_venues)


new_jersey_venues = getNearbyVenues1(name = 'Jersey City', latitudes = lat_newJercy ,longitudes = long_newJercy, radius = 3500)
new_jersey_venues.head(25)

,City,Latitude,Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Jersey City,38.3539,-121.9728,The Grind Shop,40.711670,-74.062872,Coffee Shop
1,Jersey City,38.3539,-121.9728,Harry’s Daughter,40.710904,-74.062071,Caribbean Restaurant
2,Jersey City,38.3539,-121.9728,Corgi Spirits at The Jersey City Distillery,40.708304,-74.064803,Distillery
3,Jersey City,38.3539,-121.9728,Hooked JC,40.714709,-74.067009,Fish & Chips Shop
4,Jersey City,38.3539,-121.9728,Liberty Science Center,40.707881,-74.055121,Science Museum
5,Jersey City,38.3539,-121.9728,O'LaLa Empanadas,40.717446,-74.072934,Latin American Restaurant
6,Jersey City,38.3539,-121.9728,The Oak On Pine,40.710312,-74.061406,Restaurant
7,Jersey City,38.3539,-121.9728,Liberty State Park,40.702635,-74.050941,State / Provincial Park
8,Jersey City,38.3539,-121.9728,99 Ranch Market,40.718434,-74.052564,Supermarket
9,Jersey City,38.3539,-121.9728,Brownstone Diner & Pancake Factory,40.716523,-74.048725,Diner


In [64]:
venues_in_newjersey = new_jersey_venues.copy(deep = True)
venues_in_newjersey.shape

(100, 7)

### Since we have got 100 categories, we are now giving weights for each category for better results

In [65]:
k = new_jersey_venues.copy(deep = True)
new_weightage_dict= {'Coffee Shop' : 3, 
'Caribbean Restaurant':3,
'Distillery':2,
'Fish & Chips Shop':3,
'Science Museum':3,
'Latin American Restaurant':4,
'Restaurant':5,
'State / Provincial Park':1,
'Diner':1,
'Supermarket':1,
'Bar':1,
'Jazz Club':1,
'Golf Course':3,
'Park':2,
'Cajun / Creole Restaurant':2,
'Bakery':2,
'Go Kart Track':3,
'Taco Place':3,
'Hot Dog Joint':2,
'Food Truck':3,
'Beer Garden':3,
'Boutique':4,
'Café':5,
'Bagel Shop':1,
'Record Shop':1,
'Bakery':1,
'Pizza Place':1,
'Ramen Restaurant':1,
'Wine Bar':3,
'Middle Eastern Restaurant':2,
'French Restaurant':2,
'Theater':2,
'Lounge':3,
'Wine Shop':3,
'Cocktail Bar':2,
'New American Restaurant':3,
'Residential Building (Apartment / Condo)':3,
'Pool':4,
'Burger Joint':5,
'Cheese Shop':1,
'Coffee Shop':1,
'Bagel Shop':1,
'Vietnamese Restaurant':1,
'Portuguese Restaurant':1,
'Ice Cream Shop':3,
'Italian Restaurant':2,
'Gym':2,
'Farmers Market':2,
'Bar':3,
'Pizza Place':3,
'Bakery':2,
'Bookstore':3,
'Bar':3,
'Farmers Market':4,
'Asian Restaurant':5,
'Tea Room':1,
'Donut Shop':1,
'Historic Site':1,
'Gym / Fitness Center':1,
'Café':1,
'Mexican Restaurant':3,
'Plaza':2,
'Gay Bar':2,
'Bar':3,
'College Administrative Building':3,
'Mexican Restaurant':2,
'Bakery':3,
'American Restaurant':3,
'American Restaurant':4,
'American Restaurant':5,
'Café':3,
'New American Restaurant':1,
'Chocolate Shop':1,
'Gym':1,
'Grocery Store':1,
'Middle Eastern Restaurant':3,
'American Restaurant':2,
'Frozen Yogurt Shop':2,
'Japanese Restaurant':2,
'Bar':3,
'Liquor Store':3,
'Ice Cream Shop':2,
'Fish Market':3,
'Indie Movie Theater':3,
'Grocery Store':4,
'Modern European Restaurant':5,
'American Restaurant':1,
'Poke Place':1,
'Ramen Restaurant':1,
'Diner':2,
'Brewery':2,
'Burger Joint':3,
'Burger Joint':2,
'Café':2,
'Fried Chicken Joint':2,
'Beer Garden':3,
'Gym / Fitness Center':3,
'Vietnamese Restaurant':2,
'Italian Restaurant':3,
'Pet Store':3}

In [66]:
import matplotlib.cm as cm
import matplotlib.colors as colors
import folium

# create map of the venues that we have using latitude and longitudes
venues_map = folium.Map(location=[lat_newJercy, long_newJercy], zoom_start=15) # generate map centred around Jersey city


# add Jersey City as a red circle mark
folium.features.CircleMarker(
    [lat_newJercy, long_newJercy],
    radius=10,
    popup='Jersey city',
    fill=True,
    color='red',
    fill_color='red',
    fill_opacity=0.6
    ).add_to(venues_map)

In [68]:
# add all the venuew of the Jersey city to the map as blue circle markers
for lat, lng, label in zip(venues_in_newjersey['Venue Latitude'], venues_in_newjersey['Venue Longitude'], venues_in_newjersey['Venue']):
    label=folium.Popup(label,parse_html=True)
    folium.features.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='red',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.6,
        parse_html = False).add_to(venues_map)
venues_map

### Assigning weights to each category, same as we gave for each city

In [70]:
# Calculating new weights for our data frame as we have given weights for all categories

allVenuesinCity1 = k['Venue Category']

f_weights1 = []
for i in allVenuesinCity1:
    if i in new_weightage_dict.keys():
        f_weights1.append(new_weightage_dict[i])
    else :
        f_weights1.append(0)
k['weights'] = f_weights1;
k.head(25)

,City,Latitude,Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category,weights
0,Jersey City,38.3539,-121.9728,The Grind Shop,40.711670,-74.062872,Coffee Shop,1
1,Jersey City,38.3539,-121.9728,Harry’s Daughter,40.710904,-74.062071,Caribbean Restaurant,3
2,Jersey City,38.3539,-121.9728,Corgi Spirits at The Jersey City Distillery,40.708304,-74.064803,Distillery,2
3,Jersey City,38.3539,-121.9728,Hooked JC,40.714709,-74.067009,Fish & Chips Shop,3
4,Jersey City,38.3539,-121.9728,Liberty Science Center,40.707881,-74.055121,Science Museum,3
5,Jersey City,38.3539,-121.9728,O'LaLa Empanadas,40.717446,-74.072934,Latin American Restaurant,4
6,Jersey City,38.3539,-121.9728,The Oak On Pine,40.710312,-74.061406,Restaurant,5
7,Jersey City,38.3539,-121.9728,Liberty State Park,40.702635,-74.050941,State / Provincial Park,1
8,Jersey City,38.3539,-121.9728,99 Ranch Market,40.718434,-74.052564,Supermarket,1
9,Jersey City,38.3539,-121.9728,Brownstone Diner & Pancake Factory,40.716523,-74.048725,Diner,2


In [71]:
# Dropping unnecessary columns 

newframe = k[['City','Venue Category','weights']].copy()
newframe = k.groupby(['Venue Category']).mean()
newframe.drop(columns = ["Latitude", "Longitude"], inplace = True)
newframe

,Venue Latitude,Venue Longitude,weights
Venue Category,,,
American Restaurant,40.714692,-74.041587,1
Asian Restaurant,40.721099,-74.044339,5
Australian Restaurant,40.717187,-74.044216,0
Bagel Shop,40.722990,-74.058068,1
Bakery,40.723192,-74.050249,3
Bar,40.718437,-74.058701,3
Beer Garden,40.715149,-74.046633,3
Bookstore,40.719984,-74.043205,3
Boutique,40.717606,-74.044299,4


### Using K Means algorithm to cluster the venues and calculating the weights for each cluster to decide which cluster would be the best area to open Best Bar

In [72]:
# Cluster them using K means algorithm 
from scipy import stats
from sklearn.cluster import KMeans
import matplotlib.pyplot as plt
import seaborn as sns
#Standardize
clmns = ['weights','Venue Latitude', 'Venue Longitude']
df_tr_std = stats.zscore(newframe[clmns])
#Cluster the data
kmeans = KMeans(n_clusters=3, random_state=0).fit(df_tr_std)
labels = kmeans.labels_
newframe['clusters'] = labels
#Add the column into our list
clmns.extend(['clusters'])
#Lets analyze the clusters
kframe = newframe[clmns].groupby(['Venue Category']).mean()
kframe = kframe.reset_index(drop = False)
kframe.head()

,Venue Category,weights,Venue Latitude,Venue Longitude,clusters
0,American Restaurant,1,40.714692,-74.041587,2
1,Asian Restaurant,5,40.721099,-74.044339,0
2,Australian Restaurant,0,40.717187,-74.044216,2
3,Bagel Shop,1,40.722990,-74.058068,2
4,Bakery,3,40.723192,-74.050249,0


In [73]:
#new group by clusters and add weights of each cluster 
finalWeight = kframe.groupby(['clusters']).mean()
finalWeight

,weights,Venue Latitude,Venue Longitude
clusters,,,
0,3.347826,40.719902,-74.047865
1,3.000000,40.707880,-74.065630
2,1.342857,40.720309,-74.047130


In [74]:
# Final coordinates of the place where we will be setting up an arcade is the one that has maximum weight for, in the above data frame
lat1 = 40.720102
long1 = -74.048121

In [77]:
# create map of the venues that we have using latitude and longitudes
final_map = folium.Map(location=[lat1, long1], zoom_start=15) # generate map centred around Jersey city


# add prefered location in the City as a green circle mark
folium.features.CircleMarker(
    [lat1, long1],
    radius=100,
    popup='Best Bar.Open soon!',
    fill=True,
    color='red',
    fill_color='green',
    fill_opacity=0.6
    ).add_to(final_map)
final_map

# Yes, we find this. Jone Doe will open his Best Bar on a corner Wayne Street ( like Bruce aka Batman) and Varick Street 